In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import scipy.sparse.linalg as spla
import sklearn
import sklearn.feature_extraction
from gensim.parsing.preprocessing import remove_stopwords, stem_text #prova su jupyter

np.set_printoptions(suppress=True) # questo comando impedisce a numpy di stampare numeri molto piccoli in notazione scientifica
vectorizer=sklearn.feature_extraction.text.CountVectorizer(min_df=1) # min_df=1 significa che consideriamo tutte le parole che compaiono almeno una volta (default=1)
documenti=['How to','ciao ciao ciao','sadkmdj jsadnd sad', 'skiivjjivd',' a d f c ciao cakee cake computer science']
X=vectorizer.fit_transform(documenti)
A=X.T

L=[]
for i in range(len(documenti)):
    L.append(remove_stopwords(documenti[i]))
L_final=[]
for i in range(len(documenti)):
    L_final.append(stem_text(L[i]))
print(L_final)

Y=vectorizer.fit_transform(L_final).toarray()
query1=['cake computer']
query_stemmed=stem_text(query1[0])
query1=[]
query1.append(query_stemmed)
query1=vectorizer.transform(query1).toarray()

(m,n)=A.shape
eu=np.array(np.zeros(n))
for i in range(n):
    eu[i]=np.linalg.norm(A[:,i],2) #calcolo la norma 2 di ogni colonna di A

An=np.dot(A,np.diag(1/eu)) #normalizzo le colonne di A

#eventualmente per calcolare la correlazione lineare devo trasformarmi in una distribuzione normale (vedi appunti). Qui per semplicità abbiamo solo normalizzato con la norma 2

coseni=np.dot(An.T,query1.T)/np.linalg.norm(query1,2) #calcolo il coseno tra le colonne di An e la query
print(coseni)


#VEDIAMO CON LA SVD

U,s,Vt=np.linalg.svd(An,full_matrices=True)
r=np.linalg.matrix_rank(An)

import matplotlib.pyplot as plt
plt.plot(s)

#creo usando solo 4 valori singolari
S=s[0:4]*np.eye(4,4)
An4=np.dot(U[:,0:4],np.dot(S,Vt[0:4,:]))
coseni4=np.dot(An4.T,query1.T)/np.linalg.norm(query1,2) #calcolo il coseno tra le colonne di An e la query
#vediamo come alcuni valori dei coseni salgono. Queste info aggiuntive POTREBBERO essere utili

As=sparse.csr_matrix(An)
u4,s4,v4t=sp.sparse.linalg.svds(As,k=4,solver='arpack') #arpack usa la bidiagonalizzazione di Lanczos
from sklearn.decomposition import TruncatedSVD
svdA=TruncatedSVD(n_components=4,n_iter=1,random_state=42)
svdA.fit(An)

valori_singolari=svdA.singular_values_